In [2]:
import numpy as np

In [3]:
def interior_point(A, b, c, x_init, lambda_init=None, mu_init=None,
                          maxit=1000, eps=1e-6, sigma=0.2):
    """
    Interior Point Method for Linear Programming (Affine Scaling).

    Args:
        A (np.array): Constraint matrix (m x n).
        b (np.array): RHS of constraints (m,).
        c (np.array): Objective coefficients (n,).
        x_init (np.array): Initial primal variables (n,).
        lambda_init (np.array): Initial equality dual variables (m,) (optional).
        mu_init (np.array): Initial inequality dual variables (n,) (optional).
        maxit (int): Maximum iterations.
        eps (float): Convergence tolerance.
        sigma (float): Scaling parameter.

    Returns:
        np.array: Optimal primal solution.
        int: Number of iterations.
    """
    # Ensure feasibility of the initial solution
    assert np.all(x_init >= 0), "Initial x must be feasible (x >= 0)"
    assert np.allclose(A @ x_init, b), "Initial x must satisfy Ax = b"
    m, n = A.shape
    assert m < n, "Number of constraints must be less than number of variables"

    # Initialize dual variables if not provided
    lambdas = lambda_init if lambda_init is not None else np.ones(m)
    mus = mu_init if mu_init is not None else np.ones(n)

    # Helper function: Unpack primal-dual variables
    def unpack(pdvars):
        return pdvars[:n], pdvars[n:n + m], pdvars[n + m:]

    # Combine primal and dual variables
    pdvars = np.concatenate([x_init, lambdas, mus])
    c = c.ravel()

    for iteration in range(maxit):
        # Unpack current primal and dual variables
        x, lambdas, mus = unpack(pdvars)

        # Compute duality measure
        duality_measure = np.dot(x, mus) / n

        # Build RHS of KKT system
        rhs = np.hstack([
            A.T @ lambdas + mus - c,   # Gradient of Lagrangian
            A @ x - b,                # Primal feasibility
            x * mus - sigma * duality_measure  # Complementarity
        ])

        # Build KKT system (Jacobian matrix)
        J = np.block([
            [np.zeros((n, n)), A.T, np.eye(n)],
            [A, np.zeros((m, m)), np.zeros((m, n))],
            [np.diag(mus), np.zeros((n, m)), np.diag(x)]
        ])

        # Solve for direction
        del_pdvars = np.linalg.solve(J, -rhs)
        dx, dl, dm = unpack(del_pdvars)

        # Check convergence
        if np.linalg.norm(del_pdvars, ord=np.inf) < eps:
            break

        # Calculate maximum allowable step size
        ts = np.hstack([-x / dx, -mus / dm])
        ts = ts[ts > 0]  # Only consider positive step sizes
        alpha = min(np.nanmin(ts), 1)  # Ensure steps do not exceed 1

        # Update primal-dual variables
        pdvars += alpha * del_pdvars

    return unpack(pdvars)[0], iteration


In [4]:
# Our test problem:
b = np.array([12, 8, 10])
c = np.array([3, 2])
A = np.array([[-1, 3], [1, 1], [2, -1]])
# Create initial search point near zero:
epsilon = 1e-3
x0 = np.ones(2)*epsilon
# Compute the slack variables:
xs = b - A@x0
x_init = np.hstack([x0,xs])
# Add slack variables
AI = np.hstack([A, np.eye(3)])
c = np.hstack([-c, np.zeros(3)])
xk, iter = interior_point(AI, b, c, x_init)

xk[:2], iter

(array([5.99999976, 2.00000015]), 13)